# Final Project for DTSA-5511
Kevin Juandi <br>
https://github.com/Chiuchiyin/Facial-Keypoints-Detection

In this project I will be working through the [Facial Kepoints Detection Kaggle Competition](https://www.kaggle.com/competitions/facial-keypoints-detection/overview) and build a convolutional nueral network (CNN) to perform facial recognition. I chose this project because I am very interested in computer vision so the more exposure I get to various aspects of computer vision the better. In this project, I will use a supervised deep learning model to identify key features in faces. The benefit of working through a Kaggle competition is the ability to (1) learn from others who have submitted open-source code to the competition, and (2) get immediate feedback from Kaggle as to the performance of my model. 

Here is the general working plan:

1) Download, Import, and Inspect the data

2) Clean the data

3) Perform EDA

4) Build an initial CNN, evaluate it against validation

5) Perform hyper parameter optimization to improve results

6) Submit best (by validation) CNN predictions to Kaggle

7) Debrief

Before running this notebook, please ensure that you have all the necessary libraries installed. Also please use a kernel with a GPU enabled, otherwise the training time will be overwhelming. Thank you for your time, and I hope you enjoy my project.

## Loading dependancies

In [49]:
%matplotlib inline
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Dense, Flatten, Dropout
from keras.models import Sequential
from keras.layers import LeakyReLU
import time
import warnings
warnings.simplefilter(action = 'ignore', category = FutureWarning)
warnings.simplefilter(action = 'ignore', category = Warning)
import sys
import shutil
sys.version

'3.11.5 (main, Sep 11 2023, 13:54:46) [GCC 11.2.0]'

In [39]:
tf.__version__

'2.15.0'

In [40]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [41]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Device:', tpu.master())
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
except:
    strategy = tf.distribute.get_strategy()
    if tf.test.is_gpu_available():
        print("GPU is available.")
        !nvidia-smi
    else:
        print("Training on CPU.")

print('Number of replicas:', strategy.num_replicas_in_sync)

AUTOTUNE = tf.data.experimental.AUTOTUNE

GPU is available.
Sun Feb 18 23:48:50 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.40.06              Driver Version: 551.23         CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3070 ...    On  |   00000000:01:00.0 Off |                  N/A |
| N/A   38C    P8             12W /  115W |     448MiB /   8192MiB |      3%      Default |
|                                         |                        |                  N/A |
+-----------------------------

2024-02-18 23:48:50.654635: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-18 23:48:50.654751: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-18 23:48:50.654792: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-18 23:48:50.655043: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-18 23:48:50.655057: I tensorflow/core/common_runtime/gpu/gpu

## Importing the data and data cleaning

In [43]:
os.getcwd()

'/mnt/e/Facial-Keypoints-Detection'

In [48]:
!kaggle competitions download -c facial-keypoints-detection

facial-keypoints-detection.zip: Skipping, found more recently modified local copy (use --force to force download)


In [52]:
#unzip the zip file
shutil.unpack_archive("facial-keypoints-detection.zip", "./data", "zip") 
print("Archive file unpacked successfully.")

Archive file unpacked successfully.


In [53]:
#Confirm contents of "./data"
os.listdir("./data")

['IdLookupTable.csv', 'SampleSubmission.csv', 'test.zip', 'training.zip']

In [56]:
#unzip the zip file
os.chdir("./data")
shutil.unpack_archive("training.zip", "./", "zip")
shutil.unpack_archive("test.zip", "./", "zip")
print("Archive file unpacked successfully.")

FileNotFoundError: [Errno 2] No such file or directory: './data'